In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

# from research_tools.gpu import get_gpus_available
# os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

Loaded environment variables from .env file.


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Optional
import torch
from research_tools.utils import set_seed

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda")

model_id = "LLM-LAT/zephyr7b-beta-rmu-lat-unlearn-wmdp-bio-cyber"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "meta-llama/Meta-Llama-3-8B"
model_id = "HuggingFaceH4/zephyr-7b-beta"

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, token=hf_access_token
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/adapter_config.json'
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/adapter_config.json'


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
from datasets import load_dataset
from relearn.datasets.utils import (
    load_dataset as local_load_dataset,
    DATASETS_DICT,
    Datasets,
)
from relearn.datasets.corpus import process as process_corpus
from relearn.datasets.mcq import process as process_mcq

dataset_config = DATASETS_DICT[Datasets.WMDP]

# retain_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

data_dir = Path("../data")

retain_train = local_load_dataset(data_dir, dataset_config["retain_files"])
retain_val = local_load_dataset(data_dir, dataset_config["val_retain_files"])

unlearn_files = dataset_config["unlearn_files"]
val_unlearn_files = dataset_config["val_unlearn_files"]

n_val_files = 4
max_length = 512

forget_train_1 = local_load_dataset(data_dir, unlearn_files[:n_val_files])
forget_train_2 = local_load_dataset(data_dir, unlearn_files[n_val_files:])

forget_val_1 = local_load_dataset(data_dir, val_unlearn_files[:n_val_files])
forget_val_2 = local_load_dataset(data_dir, val_unlearn_files[n_val_files:])

forget_train_1_records = process_corpus(forget_train_1, tokenizer, max_length)
forget_train_2_records = process_corpus(forget_train_2, tokenizer, max_length)
retain_train_records = process_corpus(retain_train, tokenizer, max_length)
forget_train_records = forget_train_1_records + forget_train_2_records

forget_train_mcq_1_records = process_mcq(
    forget_val_1, tokenizer, max_length, expand_choices=False
)
forget_train_mcq_2_records = process_mcq(
    forget_val_2, tokenizer, max_length, expand_choices=False
)
forget_train_mcq_records = forget_train_mcq_1_records + forget_train_mcq_2_records

forget_val_1_records = process_mcq(forget_val_1, tokenizer, max_length)
forget_val_2_records = process_mcq(forget_val_2, tokenizer, max_length)
retain_val_records = process_mcq(retain_val, tokenizer, max_length)
forget_val_records = forget_val_1_records + forget_val_2_records

Filter:   0%|          | 0/1884 [00:00<?, ? examples/s]

Map:   0%|          | 0/1884 [00:00<?, ? examples/s]

Filter:   0%|          | 0/471 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
eval_dict = {
    "A": forget_val_1_records,
    "B": forget_val_2_records,
    "retain": retain_val_records,
}

In [ ]:
import wandb

wandb.login()

config = {
    "model_id": model_id,
    "magnitude": 6.5,
    "lr": 1e-5,
    "forget_alphas": {"A": 0.5},
    "retain_alphas": {"B": 64, "retain": 1},
    "datasets_config": dataset_config,
}

run = wandb.init(project="relearn", config=config, tags=["rmu", "debug"])


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 12tqian (12tqian_team). Use `wandb login --relogin` to force relogin


In [ ]:
from relearn.unlearn.rmu import train_rmu

# i retain using corpus because everything in unlearning should be corpus, no mcq that doesn't make sense
# mcq should only be used to eval
# i care if the model can speak english, not if it can answer mcq
# force norm to be big so it's random things for weird stuff

config = run.config

model = train_rmu(
    model,
    {"A": forget_train_1_records},
    {"B": forget_train_2_records, "retain": retain_train_records},
    eval_records_dict=eval_dict,
    n_epochs=12,
    magnitude=config["magnitude"],
    lr=config["lr"],
    forget_alphas=config["forget_alphas"],
    retain_alphas=config["retain_alphas"],
    eval_at_start=False,
    max_batches=None,
    verbose=True,
    debug=False,
    tokenizer=tokenizer,
)

  0%|                                                                                                                                                                                                              | 0/118 [00:00<?, ?it/s]/mnt/align1_drive/tcqian/unlearning_order/src/relearn/unlearn/rmu/unlearn.py:252: UserWarning: Using a target size (torch.Size([1, 1, 4096])) that is different to the input size (torch.Size([4, 512, 4096])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  forget_loss = torch.nn.functional.mse_loss(
 23%|█████████████████████████████████████████████                                                                                                                                                        | 90/393 [00:06<00:42,  7.15it/s]

In [7]:
# from relearn.unlearn.rmu import train_rmu

# # i retain using corpus because everything in unlearning should be corpus, no mcq that doesn't make sense
# # mcq should only be used to eval
# # i care if the model can speak english, not if it can answer mcq
# assert False
# model = train_rmu(
#     model,
#     {"A": forget_train_1_records},
#     {"B": forget_train_2_records, "retain": retain_train_records},
#     eval_records_dict=eval_dict,
#     n_epochs=6,
#     magnitude=6.5,
#     lr=1e-5,
#     forget_alphas={
#         "A": 1,
#     },
#     retain_alphas={
#         "B": 32,
#         "retain": 1,
#     },
#     eval_at_start=True,
#     max_batches=1000000,
# )

In [8]:
# text = "Who is the 44th president of the United States?"

# # Generate text
# inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
# output = model.generate(
#     **inputs,
#     do_sample=True,
#     max_length=100,
#     num_return_sequences=5,
#     temperature=0.7,
# )

# for i, o in enumerate(output):
#     print(f"Output {i+1}: {tokenizer.decode(o, skip_special_tokens=True)}")